In [ ]:
# Importación de Librerías
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split, KFold
from  sklearn.model_selection import ParameterGrid
from sklearn.metrics import cohen_kappa_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
############### lidiar mejor con los datos
train = train.select_dtypes(exclude=['object']).dropna()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train.select_dtypes(exclude=['object']).drop("AdoptionSpeed", axis=1), 
    train.AdoptionSpeed, random_state=1, shuffle=True
)
kf = KFold(n_splits=5, random_state=1)

In [ ]:
X_train

In [ ]:
print(f"X_train: {X_train.shape}")
print(f"X_valid: {X_valid.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_valid: {y_valid.shape}")

In [ ]:

folds = [(train.iloc[train_idx].index, train.iloc[valid_idx].index) for train_idx, valid_idx in kf.split(train)]
folds[0]

In [ ]:
folds[0][0].shape, folds[0][1].shape

In [ ]:
folds[0][0]

In [ ]:
depths = list(range(2, 40, 2)) + [None]
depths

In [ ]:
models = [
    {
     "algorithm":LGBMClassifier, "params": ParameterGrid({'num_leaves': [10, 20], 'min_child_samples': [20, 40, 100]})
    },
    {
    "algorithm": RandomForestClassifier, "params": ParameterGrid({'n_estimators': [50, 100, 150], 'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22]})
    }
# from xgboost import XGBClassifier
]


In [ ]:
res = pd.DataFrame([], index=[p for m in models for p in m['params']], columns=[ "fold_" + str(i) for i in range(len(folds))] + [ "valid"])

In [ ]:
res

In [ ]:
def metric(y_true, y_pred):
    res = cohen_kappa_score(y_true, y_pred, weights= 'quadratic')
    return "kappa", res, True

In [ ]:
for model_specs in models:
    #res["algo"] = model_specs
    for params in model_specs["params"]:
        valid_probs = []
        for i, (train_idx, valid_idx) in enumerate(folds):
            X_train = train.loc[train_idx]
            y_train = X_train.AdoptionSpeed
            X_train = X_train.drop("AdoptionSpeed", axis=1)
            X_valid = train.loc[valid_idx]
            y_valid = X_valid.AdoptionSpeed
            X_valid = X_valid.drop("AdoptionSpeed", axis=1)
            #print(3)
            learner = model_specs["algorithm"](**params)
            learner.fit(X_train, y_train)
            probs = pd.DataFrame(learner.predict_proba(X_valid), index=X_valid.index)
            #probs=learner.predict_proba(X_valid)
            #print(probs.shape)
            #print(4)
            valid_probs.append(probs)
            res.loc[str(params), "fold_" + str(i)] = roc_auc_score(y_valid.loc[probs.index], probs,  multi_class="ovo")
            #break
        #print(5)
        valid_probs = pd.concat(valid_probs)
        res.loc[str(params), "valid"] = roc_auc_score(train.loc[valid_probs.index, "AdoptionSpeed"], valid_probs,  multi_class="ovo")
    #print(6)

In [ ]:
res

https://scikit-learn.org/stable/modules/classes.html#hyper-parameter-optimizers